# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 3. Klasifikasi K-Nearest Neighbor (KNN)


Tuliskan Nama, NIM, dan kelas Anda:

Nama  : Salma Adzra Fathina

NIM   : 225150207111042

Kelas : PPM - F

### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [126]:
! wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2024-03-17 13:33:15--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv.5’

iris.csv.5          100%[===================>]   3.88K  --.-KB/s    in 0s      

2024-03-17 13:33:15 (61.0 MB/s) - ‘iris.csv.5’ saved [3975/3975]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [127]:
import pandas as pd
import numpy as np

data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [128]:
data.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [129]:
from sklearn.model_selection import train_test_split
data_latih, data_uji = train_test_split(data, test_size = 0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [130]:
print(data_uji.shape[0])
print(data_latih.shape[0])

30
120


Pisahkan label/kategori dari data latih dan data uji menjadi variabel tersendiri. Beri nama variabelnya **label_latih** dan **label_uji**

In [131]:
label_latih = data_latih.pop('variety')
label_uji = data_uji.pop('variety')

In [132]:
data_uji.head()

,sepal.length,sepal.width,petal.length,petal.width
50,7.0,3.2,4.7,1.4
40,5.0,3.5,1.3,0.3
138,6.0,3.0,4.8,1.8
122,7.7,2.8,6.7,2.0
22,4.6,3.6,1.0,0.2


In [133]:
data_latih.head()

,sepal.length,sepal.width,petal.length,petal.width
105,7.6,3.0,6.6,2.1
58,6.6,2.9,4.6,1.3
56,6.3,3.3,4.7,1.6
41,4.5,2.3,1.3,0.3
27,5.2,3.5,1.5,0.2


In [134]:
print(label_latih)

105     Virginica
58     Versicolor
56     Versicolor
41         Setosa
27         Setosa
          ...    
104     Virginica
5          Setosa
10         Setosa
147     Virginica
38         Setosa
Name: variety, Length: 120, dtype: object


In [135]:
print(label_uji)

50     Versicolor
40         Setosa
138     Virginica
122     Virginica
22         Setosa
146     Virginica
65     Versicolor
124     Virginica
98     Versicolor
13         Setosa
11         Setosa
85     Versicolor
16         Setosa
67     Versicolor
45         Setosa
121     Virginica
119     Virginica
6          Setosa
81     Versicolor
70     Versicolor
2          Setosa
8          Setosa
60     Versicolor
109     Virginica
135     Virginica
54     Versicolor
86     Versicolor
72     Versicolor
97     Versicolor
99     Versicolor
Name: variety, dtype: object


## 3) Menghitung jarak euclidean

Tahapan awal dari algoritma KNN adalah perhitungan jarak. Salah satu metode perhitungan jarak yang bisa digunakan adalah jarak Euclidean. Buatlah fungsi bernama **jarakEu** yang berfungsi menghitung jarak euclidean dari dua buah vektor (tanpa kelas)

In [136]:
def jarakEu(data1, data2):
  jarak = np.square(data1-data2)
  jarak = np.sum(jarak)
  return np.sqrt(jarak)

Uji fungsi **jarakEu** untuk menghitung jarak antara data latih pertama dengan data uji pertama.

In [137]:
jarak = jarakEu(data_latih.iloc[0], data_uji.iloc[0])
print(jarak)

2.121320343559642


## 4) Algoritma KNN

Implementasikan algoritma KNN dengan tahapan-tahapan berikut :
1. Hitung jarak antara data uji dengan setiap data latih.
2. Cari *k* data latih dengan jarak terkecil.
3. Tentukan kelas dari *k* data latih tersebut.
4. Kelas data uji ditentukan dari mayoritas kelas *k* data latih.

Buatlah fungsi dengan nama **knn** yang menerima input berupa *k*, sekumpulan data latih dan labelnya, serta sebuah data uji

In [138]:
from collections import Counter
def knn (k, datalatih, labellatih, datauji):
  jarak = np.array([jarakEu(datalatih.iloc[x], datauji) for x in range(datalatih.shape[0])])
  indeks_k_minimum = jarak.argsort()[:k]
  k_kelas = labellatih.iloc[indeks_k_minimum].to_numpy()
  counter = Counter(k_kelas)
  kelas_uji = counter.most_common(1)[0][0]
  return kelas_uji

Lakukan pengujian fungsi **knn** untuk menentukan kelas dari data uji pertama

In [139]:
hasil = knn(3, data_latih, label_latih, data_uji.iloc[0])
print(hasil)

Versicolor


## 5) KNN pada Scikit-learn

Library scikit-learn memiliki algoritma KNN yang siap untuk digunakan. Sebagai pengenalan, Anda akan melakukan klasifikasi data yang sama menggunakan KNN yang tersedia pada library scikit-learn

In [161]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors= 3)
KNN.fit(data_latih, label_latih)
kelas = KNN.predict([data_uji.iloc[0]])
print(kelas)

['Versicolor' 'Setosa' 'Virginica' 'Virginica' 'Setosa' 'Virginica'
 'Versicolor' 'Virginica' 'Versicolor' 'Setosa' 'Setosa' 'Versicolor'
 'Setosa' 'Versicolor' 'Setosa' 'Virginica' 'Versicolor' 'Setosa'
 'Versicolor' 'Virginica' 'Setosa' 'Setosa' 'Versicolor' 'Virginica'
 'Virginica' 'Versicolor' 'Versicolor' 'Virginica' 'Versicolor'
 'Versicolor']


## TUGAS
1. Buatlah fungsi bernama **jarakCity** yang berfungsi menghitung jarak City Block antara dua vektor.
2. Buatlah sebuah fungsi bernama **knn_multi** yang dapat menentukan kelas dari **semua** data uji (Fungsi **knn** yang sudah Anda buat hanya dapat menentukan kelas dari sebuah data uji. Anda dapat memanfaatkan fungsi **knn** yang telah ada
3. Buatlah fungsi bernama **cek_hasil** yang berfungsi membandingkan hasil klasifikasi dengan label sebenarnya. Fungsi ini menerima input 2 buah list, yaitu list label hasil klasifikasi dan list label data uji yang sebenarnya. Output dari fungsi ini adalah integer, yang menunjukkan berapa banyak label hasil klasifikasi yang sama dengan label data uji sebenarnya.
4. Jalankan fungsi **knn_multi** dengan nilai k=3, k=7,k=9 dan k=13. Amati hasil dari fungsi **cek_hasil** dengan nilai k tersebut. Apa kesimpulan Anda?

In [141]:
#jawaban soal 1
def jarakCity(city1, city2):
  jarak = np.sum(np.absolute(city1- city2))
  return jarak

In [142]:
jarak = jarakCity(data_latih.iloc[0], data_uji.iloc[0])
print(jarak)

3.3999999999999995


In [188]:
#jawaban soal 2
def knn_multi(k, datalatih, labellatih, datauji):
  hasil = []
  for i in range(len(datauji)):
    jarak = np.array([jarakCity(datalatih.iloc[x], datauji.iloc[i]) for x in range(datalatih.shape[0])])
    indeks_k_minimum = jarak.argsort()[:k]
    k_kelas = labellatih.iloc[indeks_k_minimum].to_numpy()
    counter = Counter(k_kelas)
    kelas_uji = counter.most_common(1)[0][0]
    hasil.append(kelas_uji)
  return hasil

In [189]:
hasil = knn_multi(3 , data_latih, label_latih, data_uji)
print(hasil)

['Versicolor', 'Setosa', 'Virginica', 'Virginica', 'Setosa', 'Virginica', 'Versicolor', 'Virginica', 'Versicolor', 'Setosa', 'Setosa', 'Versicolor', 'Setosa', 'Versicolor', 'Setosa', 'Virginica', 'Versicolor', 'Setosa', 'Versicolor', 'Virginica', 'Setosa', 'Setosa', 'Versicolor', 'Virginica', 'Virginica', 'Versicolor', 'Versicolor', 'Virginica', 'Versicolor', 'Versicolor']


In [190]:
#apabila menggunakan library Scikit-learn :D

from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors= 3)
KNN.fit(data_latih, label_latih)
kelas = KNN.predict(data_uji)
print(kelas)

['Versicolor' 'Setosa' 'Virginica' 'Virginica' 'Setosa' 'Virginica'
 'Versicolor' 'Virginica' 'Versicolor' 'Setosa' 'Setosa' 'Versicolor'
 'Setosa' 'Versicolor' 'Setosa' 'Virginica' 'Versicolor' 'Setosa'
 'Versicolor' 'Virginica' 'Setosa' 'Setosa' 'Versicolor' 'Virginica'
 'Virginica' 'Versicolor' 'Versicolor' 'Virginica' 'Versicolor'
 'Versicolor']


Jadi, hasil dari prediksi kelas tiap data uji adalah:
['Versicolor', 'Setosa', 'Virginica', 'Virginica', 'Setosa', 'Virginica', 'Versicolor', 'Virginica', 'Versicolor', 'Setosa', 'Setosa', 'Versicolor', 'Setosa', 'Versicolor', 'Setosa', 'Virginica', 'Versicolor', 'Setosa', 'Versicolor', 'Virginica', 'Setosa', 'Setosa', 'Versicolor', 'Virginica', 'Virginica', 'Versicolor', 'Versicolor', 'Virginica', 'Versicolor', 'Versicolor']





In [191]:
#jawaban soal 3
def cek_hasil(list_label_hasil, list_label_uji):
  hasil = np.sum(np.array(list_label_hasil) == np.array(list_label_uji))
  return hasil

In [192]:
sama = cek_hasil(hasil, label_uji)
print(sama)

27


In [182]:
#mari kita cek label aslinya dari data uji :D
print(label_uji)

50     Versicolor
40         Setosa
138     Virginica
122     Virginica
22         Setosa
146     Virginica
65     Versicolor
124     Virginica
98     Versicolor
13         Setosa
11         Setosa
85     Versicolor
16         Setosa
67     Versicolor
45         Setosa
121     Virginica
119     Virginica
6          Setosa
81     Versicolor
70     Versicolor
2          Setosa
8          Setosa
60     Versicolor
109     Virginica
135     Virginica
54     Versicolor
86     Versicolor
72     Versicolor
97     Versicolor
99     Versicolor
Name: variety, dtype: object


Dapat disimpulkan bahwa hasil prediksi label pada data uji menggunakan algoritma KNN, terdapat 3 dari 30 label dengan prediksi yang tidak tepat

yaitu:

1.   index ke-16, dimana pada prediksi saya adalah versicolor. Label yang benar adalah virginica
2.   index ke-19, dimana pada prediksi saya adalah virginica. Label yang benar adalah versicolor
3.   index ke-27, dimana pada prediksi saya adalah virginica. Label yang benar adalah versicolor

In [206]:
#jawaban soal 4
k = [3, 7, 9, 13]
for k in k:
  hasil = knn_multi(k , data_latih, label_latih, data_uji)
  sama = cek_hasil(hasil, label_uji)
  print(f"Untuk k={k}, jumlah label yang sama ada {sama} label")

Untuk k=3, jumlah label yang sama ada 27 label
Untuk k=7, jumlah label yang sama ada 29 label
Untuk k=9, jumlah label yang sama ada 28 label
Untuk k=13, jumlah label yang sama ada 28 label


jumlah sampel data latih terdekat dewngan data uji yang diambil (atau disebut dengan k), akan menentukan kelas dari tiap data uji berdasarkan kelas mayoritas dari tiap-k sampel tersebyt. Jadi, ketika ktia mengubah nilai k, kita sebenarnya mengubah jumlah tetangga yang dihitung oleh algoritma saat membuat prediksi.

*   Jika k terlalu kecil, model mungkin menjadi terlalu sensitif terhadap noise atau variasi acak dalam data, sehingga prediksi yang kurang akurat.
*   Jika k terlalu besar, model mungkin menjadi terlalu umum, sehingga algoritma mungkin tidak mempertimbangkan fitur penting dalam data, dan ini juga bisa mengurangi akurasi prediksi.

Oleh karena itu kita perlu mengoptimalkan nilai k dengan algoritma genetika. Cukup riweuh yach